In [79]:
import numpy as np 
import matplotlib.pyplot as plt 
import cv2
import os
import mediapipe as mp
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from keras.applications.vgg16 import VGG16

In [72]:
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [73]:
path = ".\\videos"
import pickle
import gzip
with gzip.open("phoenix14t.pami0.test.annotations_only.gzip", 'rb') as f:
    annotations = pickle.load(f)

print(annotations[0])

{'name': 'test/25October_2010_Monday_tagesschau-17', 'signer': 'Signer01', 'gloss': 'REGEN SCHNEE REGION VERSCHWINDEN NORD REGEN KOENNEN REGION STERN KOENNEN SEHEN', 'text': 'regen und schnee lassen an den alpen in der nacht nach im norden und nordosten fallen hier und da schauer sonst ist das klar .'}


In [74]:
x_train = []
y_train = []

for annot in annotations:
    temp = annot['name'].split('/')
    annot['name'] = '\\'.join(temp)
    dest = os.path.join(path,annot['name'] + '.mp4')
    x_train.append(dest)
    y_train.append(annot['text'])

x_train = np.array(x_train)
y_train = np.array(y_train)
print(x_train.shape,y_train.shape)  

(642,) (642,)


In [75]:
print(x_train[0])

.\videos\test\25October_2010_Monday_tagesschau-17.mp4


In [5]:
# mp_holistic = mp.solutions.holistic
# mp_drawing = mp.solutions.drawing_utils

In [6]:
# def mediapipe_detection(image,model):
#     image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
#     image.flags.writeable = False
#     results = model.process(image)
#     image.flags.writeable = True
#     image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

#     return image,results

In [7]:
# def draw_landmarks(image,results):
#     mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_CONTOURS)
#     mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
#     mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
#     mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [8]:
# def draw_styled_landmarks(image,results):
#     mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_CONTOURS,
#                             mp_drawing.DrawingSpec(color = (80,110,10),thickness=1,circle_radius=1),
#                             mp_drawing.DrawingSpec(color = (80,256,121),thickness=1,circle_radius=1),
#                             )
#     mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
#                             mp_drawing.DrawingSpec(color = (80,22,10),thickness=2,circle_radius=4),
#                             mp_drawing.DrawingSpec(color = (80,44,121),thickness=2,circle_radius=2),
#                             )
#     mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
#                             mp_drawing.DrawingSpec(color = (121,22,76),thickness=2,circle_radius=4),
#                             mp_drawing.DrawingSpec(color = (121,44,250),thickness=2,circle_radius=2),
#                             )
#     mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
#                             mp_drawing.DrawingSpec(color = (245,117,66),thickness=2,circle_radius=4),
#                             mp_drawing.DrawingSpec(color = (245,66,230),thickness=2,circle_radius=2),
#                             )

In [9]:
# def extract_keypoints(results):
#     pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
#     face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
#     lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
#     rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

#     return np.concatenate([pose,face,lh,rh])

In [40]:
# image_data = []
# for i in range(10):

#     video_data = []

#     cap = cv2.VideoCapture(x_train[i])

#     with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
#         while(True):
#             ret,frame = cap.read()
#             if(ret==False):
#                 break
            
#             image,results = mediapipe_detection(frame,holistic)

#             frame_data = extract_keypoints(results)
#             video_data.append(frame_data)

#             # draw_styled_landmarks(image,results)
            
#             # cv2.imshow('Video Feed',image)

#             if cv2.waitKey(10) & 0xFF == ord('q'):
#                 break

#         cap.release()
#         cv2.destroyAllWindows()

    
#     total = len(video_data)
#     frames = 40
#     interval = total/frames
#     f_no = 0
#     video_data = np.array(video_data)

#     extarcted_frames = []
#     for i in range(total):
#         if(i==int(f_no*interval)):
#             extarcted_frames.append(video_data[i])
#             f_no += 1
#     extracted_frames = np.array(extarcted_frames)

#     print(extracted_frames.shape)
    
#     image_data.append(extracted_frames)

(40, 1662)
(40, 1662)
(40, 1662)
(40, 1662)
(40, 1662)
(40, 1662)
(40, 1662)
(40, 1662)
(40, 1662)
(40, 1662)


In [41]:
# from gensim.models import word2vec

# sentences = [
#     'He is playing in the field.',
#     'He is running towards the football.',
#     'The football game ended.',
#     'It started raining while everyone was playing in the field.'
# ]

# for i, sentence in enumerate(sentences):
# 	tokenized= []
# 	for word in sentence.split(' '):
# 		word = word.split('.')[0]
# 		word = word.lower()
# 		tokenized.append(word)
# 	sentences[i] = tokenized

# model = word2vec.Word2Vec(sentences, workers = 1, min_count = 1, window = 3, sg = 0)
# similar_word = model.wv.most_similar('football')[0]
# print("Most common word to football is: {}".format(similar_word[0]))


In [83]:
image_data = []
size = 2
for i in range(size):

    video_data = []

    cap = cv2.VideoCapture(x_train[i])

    while(True):
        ret,frame = cap.read()
        if(ret==False):
            break
            
        video_data.append(frame)

            # draw_styled_landmarks(image,results)
            
            # cv2.imshow('Video Feed',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    
    total = len(video_data)
    frames = 40
    interval = total/frames
    f_no = 0
    video_data = np.array(video_data)

    extracted_frames = []
    for i in range(total):
        if(i==int(f_no*interval)):
            temp = cv2.resize(video_data[i],(224,224))
            extracted_frames.append(temp)
            f_no += 1
    extracted_frames = np.array(extracted_frames)

    print(extracted_frames.shape)
    
    image_data.append(extracted_frames)

(40, 224, 224, 3)
(40, 224, 224, 3)


In [84]:
def model_cnn_load():
    model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    out = model.layers[-2].output
    model_final = Model(inputs=model.input, outputs=out)
    return model_final

In [85]:
model = model_cnn_load()
sign_language = []
for i in range(len(image_data)):
    features = model.predict(image_data[i],batch_size=128)
    print(features.shape)
    sign_language.append(features)
print(len(sign_language))

553476096/553467096 [==============================] - 230s 0us/step
(40, 4096)
(40, 4096)
2


In [88]:
sign_lang = np.array(sign_language)
print(sign_lang.shape)

(2, 40, 4096)


## Word Embedding

In [97]:
print(y_train.shape)

(642,)


In [98]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [99]:
for i in range(y_train.shape[0]):
    y_train[i] = "<bos>" +  y_train[i] + "<eos>"

In [100]:
# Tokenize words
ger_text_tokenized, ger_text_tokenizer = tokenize(y_train[:2])

print('Maximum length german sentence: {}'.format(len(max(ger_text_tokenized,key=len))))

# Check language length
german_vocab = len(ger_text_tokenizer.word_index) + 1

print("german vocabulary is of {} unique words".format(german_vocab))

Maximum length german sentence: 28
german vocabulary is of 37 unique words


In [101]:
max_sentence_length = len(max(ger_text_tokenized,key=len))
ger_pad_sentence = pad_sequences(ger_text_tokenized, padding = "post", truncating='post')

## Model

In [60]:
inp_dim = sign_language.shape[0]*sign_language.shape[1]

In [66]:
sign_language = sign_language.reshape(10,-1)

In [67]:
print(sign_language.shape)

(10, 66480)


In [68]:
input_sequence = Input(shape=sign_language.shape[1])
embedding = Embedding(input_dim=inp_dim, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_sentence_length)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(german_vocab))(decoder)

In [69]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 66480)]           0         
                                                                 
 embedding_1 (Embedding)     (None, 66480, 128)        51200     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVector  (None, 24, 64)           0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 24, 64)            33024     
                                                                 
 time_distributed (TimeDistr  (None, 24, 105)          6825      
 ibuted)                                                     

In [59]:
def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 5

print("The german sentence is: {}".format(y_train[index]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(sign_language[index])[0], ger_text_tokenizer))

105
